## SentenceBERT

In [ ]:
import pandas as pd

In [ ]:
# Read csv file train.csv and store it in a variable called train_data
train_data = pd.read_csv('../input/feedback-prize-effectiveness/train.csv')
train_data.head()

In [ ]:
# create a list of sentences from the train_data - train_data['discourse_text']
train_sentences = train_data['discourse_text'].tolist()

discourse_type = train_data['discourse_type'].tolist()
# discourse_type

In [ ]:
import csv

def readCSV(filename):
    rows = []
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        for row in csvreader:
            rows.append(row)
    return rows

In [ ]:
discourses = []
dis_types = []
dis_effectiveness = []
rows = readCSV("../input/feedback-prize-effectiveness/train.csv")
for row in rows:
    if len(row[2])<=510:
        discourses.append(row[2])
        dis_types.append(row[3])
        dis_effectiveness.append(row[4])
    while len(row[2])>510:
        temp = row[2][:510]
        row[2] = row[2][510:]
        discourses.append(temp)
        dis_types.append(row[3])
        dis_effectiveness.append(row[4])
print(len(discourses),discourses[0])
print(len(dis_types))
print(len(dis_effectiveness))

In [ ]:
typeDict = {'Lead':0,'Position':1,'Claim':2,'Counterclaim':3,'Rebuttal':4,'Evidence':5,'Concluding Statement':6}
effectDict = {'Ineffective':0,'Adequate':1,'Effective':2}

In [ ]:
import numpy as np
import os
import gensim
import torch
import transformers as ppb
# from sent2vec.splitter import *

import os
import re
import spacy

ROOT_DIR = os.getcwd() 
TEST_DIR = os.path.join(ROOT_DIR, 'test')
DATA_DIR = os.path.join(TEST_DIR, 'dataset')

ROOT_LOCAL = '/Users/pedramataee/'
ROOT_LINUX = '~'

WIKI_PATH = os.path.join(ROOT_LINUX, 'gensim-data/glove-wiki-gigaword-300') 
PRETRAINED_VECTORS_PATH_WIKI = os.path.join(WIKI_PATH, 'glove-wiki-gigaword-300.gz')

FASTTEXT_NEWS_PATH = os.path.join(ROOT_LINUX, 'gensim-data/fasttext-wiki-news-subwords-300') 
PRETRAINED_VECTORS_PATH_FASTTEXT = os.path.join(FASTTEXT_NEWS_PATH, 'fasttext-wiki-news-subwords-300.gz')

# Make sure to download "en_core_web_sm" package on your machine.
# In case, you can run: "python3 -m spacy download en_core_web_sm"
os.environ['LANGUAGE_MODEL_SPACY'] = "en_core_web_sm"


class Splitter:
    def __init__(self):
        self.words = []
        self.sentences = []
        try:
            self.nlp = spacy.load(os.environ['LANGUAGE_MODEL_SPACY'])
        except Exception as error:
            print(f'{error}\n\n Install "en_core_web_sm" in your environment. '
                  f'Try running: "python3 -m spacy download en_core_web_sm" \n '
                  f'or follow instrucions here: https://spacy.io/usage')

    def sent2words(self, sentences, **kwargs):
        add_stop_words = kwargs.get('add_stop_words', [])
        remove_stop_words = kwargs.get('remove_stop_words', [])

        for w in add_stop_words:
            self.nlp.vocab[w].is_stop = True
        for w in remove_stop_words:
            self.nlp.vocab[w].is_stop = False

        words = []
        for sentence in sentences:
            doc = self.nlp(sentence.lower())
            words.append([token.lemma_ for token in doc if not token.is_punct | token.is_space | token.is_stop])

        self.words = words

    def text2sents(self, texts):
        for text in texts:
            doc = self.nlp(text)
            span = doc[0:5]
            sents = list(doc.sents)
            self.sentences.extend([sent for sent in sents])

    def text2words(self, texts):
        doc = self.nlp(texts)
        tokenized_texts = []
        for w in doc:
            is_clean = w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num
            if is_clean:
                tokenized_texts.append(w.lemma_)

        self.words = tokenized_texts


def sentencizer_by_regex(texts):
    alphabets = "([A-Za-z])"
    prefixes = r"(Mr|St|Mrs|Ms|Dr)[.]"
    suffixes = r"(Inc|Ltd|Jr|Sr|Co|etc)"
    starters = r"(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
    acronyms = r"([A-Z][.][A-Z][.](?:[A-Z][.])?)"
    websites = r"[.](com|net|org|io|gov)"

    text = " " + texts + "  "
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)
    if "Ph.D" in text:
        text = text.replace("Ph.D.", "Ph<prd>D<prd>")
    text = re.sub(r"\s" + alphabets + "[.] ", " \\1<prd> ", text)
    text = re.sub(acronyms + " " + starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>\\3<prd>", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>", text)
    text = re.sub(" " + suffixes + "[.] " + starters, " \\1<stop> \\2", text)
    text = re.sub(" " + suffixes + "[.]", " \\1<prd>", text)
    text = re.sub(" " + alphabets + "[.]", " \\1<prd>", text)
    if "”" in text:
        text = text.replace(".”", "”.")
    if "\"" in text:
        text = text.replace(".\"", "\".")
    if "!" in text:
        text = text.replace("!\"", "\"!")
    if "?" in text:
        text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

class Vectorizer:
    """
    pretrained_weights: str, default='distilbert-base-uncased'

        If the string does not include an extension .txt, .gz or .bin, then Bert vectorizer is loaded using the specified weights.
        Example: pass 'distilbert-base-multilingual-cased' to load Bert base multilingual model.

        To load word2vec vectorizer pass a valid path to the weights file (.txt, .gz or .bin).
        Example: pass 'glove-wiki-gigaword-300.gz' to load the Wiki vectors (when saved in the same folder you are running the code).

    
    ensemble_method: str, default='average'

        How word vectors are computed into sentece vectors.
    
    """
    def __init__(self, pretrained_weights = 'distilbert-base-uncased',
                       ensemble_method = 'average'):
        _, ext = os.path.splitext(pretrained_weights)
        self.vectors = []
        if not ext:
            print(f'Initializing Bert {pretrained_weights}')
            self.vectorizer = BertVectorizer(pretrained_weights=pretrained_weights)
        else:
            print(f'Initializing word2vec with vector path {pretrained_weights}')
            self.vectorizer = GensimVectorizer(pretrained_weights=pretrained_weights, 
                                               ensemble_method=ensemble_method)

    def run(self, sentences, remove_stop_words = ['not'], add_stop_words = []):
        # SANITY CHECK
        assert type(sentences) == list, 'A list must be passed!'
        for sentence in sentences:
            if type(sentence) != str:
                raise TypeError(f'All items must be string type but {sentence} is type {type(sentence)}.')
        # RUN
        self.vectorizer._execute(sentences, remove_stop_words=remove_stop_words, add_stop_words=add_stop_words)
        vectors = self.vectorizer.vectors
        for idx in range(vectors.shape[0]):
            self.vectors.append(vectors[idx])


class BaseVectorizer():
    def __init__(self, **kwargs):
        self.pretrained_weights = kwargs.get('pretrained_weights')
        self.ensemble_method = kwargs.get('ensemble_method')
        self.vectors = []
    
    def _load_model(self):
        pass

    def _execute(self):
        pass


class BertVectorizer(BaseVectorizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._load_model()
    
    def _load_model(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'Vectorization done on {self.device}')
        model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel,
                                                            ppb.DistilBertTokenizer,
                                                            self.pretrained_weights)
        self.tokenizer = tokenizer_class.from_pretrained('../input/huggingface-bert/bert-base-uncased',local_files_only = True)
        self.model = model_class.from_pretrained('../input/huggingface-bert/bert-base-uncased',local_files_only = True)
    
    def _execute(self, sentences, **kwargs):
        model = self.model.to(self.device)
        model.eval()
        tokenized = list(map(lambda x: self.tokenizer.encode(x, add_special_tokens=True), sentences))
        max_len = 0
        for i in tokenized:
            if len(i) > max_len:
                max_len = len(i)
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        # Move inputs to same device as model
        input_ids = torch.tensor(np.array(padded)).type(torch.LongTensor).to(self.device)
        with torch.no_grad():
            last_hidden_states = model(input_ids)
        # Move vector results back to cpu if calculation was done on GPU
        vectors = last_hidden_states[0][:, 0, :].cpu().numpy()
        self.vectors = vectors

class GensimVectorizer(BaseVectorizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._load_model()
    
    def _load_model(self):
        _, file_extension = os.path.splitext(self.pretrained_weights)
        # Checks if file extension is binary
        if file_extension == '.bin':
            self.model = gensim.models.KeyedVectors.load_word2vec_format(self.pretrained_weights, binary=True)
        elif file_extension == '.txt' or '.gz':
            self.model = gensim.models.KeyedVectors.load_word2vec_format(self.pretrained_weights)
        else:
            raise IOError(f'The file extension {file_extension} is not valid. Word2vec valid formats are ".txt" and ".bin".')
    
    def _execute(self, sentences, **kwargs):
        splitter = Splitter()
        splitter.sent2words(sentences, remove_stop_words=kwargs.get('remove_stop_words'), add_stop_words=kwargs.get('add_stop_words'))
        words = splitter.words
        vectors = []
        for element in words:
            temp = []
            for w in element:
                temp.append(self.model[w])
            if self.ensemble_method == 'average':
                element_vec = np.mean(temp, axis=0)
                try:
                    vectors = np.vstack([vectors, element_vec])
                except:
                    vectors = element_vec
        self.vectors = vectors
        

In [ ]:
from transformers import logging
logging.set_verbosity_error()

In [ ]:
# from sent2vec.vectorizer import Vectorizer
import numpy as np

def getVectors(discourses, dis_types):
    vectorizer = Vectorizer()
    dis_vectors = []
    for i in range(0,len(discourses),100):
        vectorizer.run(discourses[i:min(i+100,len(discourses))], remove_stop_words=[], add_stop_words=[])
        temp = vectorizer.vectors
        for t in range(i,min(i+100,len(discourses))):
            dis_vectors.append(temp[t][:100])
    dis_vectors = np.array(dis_vectors)

    # concatenate the type[dis_types[i]] and dis_vectors
    dis_vectors = np.concatenate((np.array([typeDict[dis_types[i]] for i in range(len(dis_types))]).reshape(-1,1),dis_vectors),axis=1)
        

    print(len(dis_vectors))
    print(dis_vectors[0])
    # return dis_vectors

    output = []
    # make the output as  the one hot encoding of the effectDict[dis_effectiveness[i]]
    for i in range(len(dis_effectiveness)):
        temp = [0,0,0]
        temp[effectDict[dis_effectiveness[i]]] = 1
        output.append(temp)
    output = np.array(output)
    print(len(output))
    print(output[0])
    return dis_vectors,output

In [ ]:
train_input, train_output = getVectors(discourses, dis_types)

In [ ]:
# take 80% of the data as training data and 20% as validation data
train_input = train_input[:int(len(train_input)*0.8)]
train_output = train_output[:int(len(train_output)*0.8)]
val_input = train_input[int(len(train_input)*0.8):]
val_output = train_output[int(len(train_output)*0.8):]

print(len(train_input))
print(len(train_output))
print(len(val_input))
print(len(val_output))

In [ ]:
val_input

In [ ]:
val_output.shape

In [ ]:
# design a tensorflow keras classfication model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

train_input = np.array(train_input)
train_output = np.array(train_output)


model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[101]),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

history = model.fit(train_input, train_output, epochs=5000)


In [ ]:
from transformers import logging

logging.set_verbosity_error()

In [ ]:
# create a submissions.csv file
# '''
# discourse_id,Ineffective,Adequate,Effective
# a261b6e14276,0.2,0.6,0.4
# 5a88900e7dc1,3.0,6.0,1.0
# 9790d835736b,1.0,2.0,3.0
# 75ce6d68b67b,0.33,0.34,0.33
# 93578d946723,0.01,0.24,0.47
# 2e214524dbe3,0.2,0.6,0.4
# 84812fc2ab9f,3.0,6.0,1.0
# c668ff840720,1.0,2.0,3.0
# 739a6d00f44a,0.33,0.34,0.33
# bcfae2c9a244,0.01,0.24,0.47
# '''

test_data = pd.read_csv('../input/feedback-prize-effectiveness/test.csv')
test_data.head()

test_sentences = test_data['discourse_text'].tolist()
test_sentences[0]

test_dis_types  = test_data['discourse_type'].tolist()

test_input,_ = getVectors(test_sentences,test_dis_types)
test_input = np.array(test_input)

test_output = model.predict(test_input)
# test_output = np.argmax(test_output, axis=1)
print(test_output)

import csv

with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["discourse_id","Ineffective","Adequate","Effective"])
    for i in range(len(test_output)):
#         temp = [0,0,0]
#         temp[test_output[i]] = 1
        writer.writerow([test_data['discourse_id'][i],test_output[i][0],test_output[i][1],test_output[i][2]])
    
    

### Predictions for the test dataset (20% of train data)

In [ ]:
# get the prediction for val_input
val_pred = model.predict(val_input)

In [ ]:
val_output

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()
print(f'Loss in test data: {loss.__call__(val_output, val_pred)}')

In [ ]:
# make the hot vector of one size (for accuracy calc)
val_pred = np.argmax(val_pred, axis=1)
val_output = np.argmax(val_output, axis=1)

In [ ]:
# get the accuracy of the model
from sklearn.metrics import accuracy_score
print(f'Accuracy: {accuracy_score(val_output, val_pred)*100} %')

In [ ]:
# get the confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(val_output, val_pred))

# get the classification report
from sklearn.metrics import classification_report
print(classification_report(val_output, val_pred))

# get the f1 score
from sklearn.metrics import f1_score
print(f1_score(val_output, val_pred, average='macro'))


In [ ]:
# get the precision score
from sklearn.metrics import precision_score
print(precision_score(val_output, val_pred, average='macro'))

# get the recall score
from sklearn.metrics import recall_score
print(recall_score(val_output, val_pred, average='macro'))

In [ ]:
# plot the confusion matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = confusion_matrix(val_output, val_pred)
df_cm = pd.DataFrame(array, index = [i for i in "012"],
                    columns = [i for i in "012"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

# plot the loss and accuracy of the model
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss = history.history['loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'g', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure(figsize=(15,10))
plt.show()

In [ ]:
print(f'Final value of Loss: {loss[-1]}')
print(f'Least value of Loss: {min(loss)}')

In [ ]:
# plot the accuracy of the model during training
plt.clf()   # clear figure

acc = history.history['accuracy']

plt.plot(epochs, acc, 'b', label='Training acc')
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure(figsize=(15,10))
plt.show()

In [ ]:
print(f'Final value of Accuracy: {acc[-1]}')
print(f'Maximum value of Accuracy: {max(acc)}')

In [ ]:
# plot the predicted and actual values of output
plt.clf()   # clear figure

# plot the val_pred and val_output arrays on the same graph
plt.plot(val_pred, 'ro', label='val_pred')
plt.plot(val_output, 'b', label='val_output')
plt.title('val_pred and val_output')
plt.xlabel('Epochs')
plt.ylabel('val_pred and val_output')
plt.legend()
plt.figure(figsize=(13,5))
plt.show()


In [ ]:
# plot the val_pred and val_output arrays
plt.hist(val_pred, bins=3, alpha=0.5, label='val_pred')
plt.hist(val_output, bins=3, alpha=0.5, label='val_output')
plt.legend(loc='upper right')
plt.figure(figsize=(10,5))
plt.show()